pip install torch torchvision transformers datasets pytesseract Pillow

In [2]:
!pip install --upgrade datasets fsspec aiohttp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.15
    Uninstalling aiohttp-3.11.15:
      Successfully uninstalled aiohttp-3.11.15
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 

In [1]:
from datasets import load_dataset

ds = load_dataset("dvgodoy/rvl_cdip_mini")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/341M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/42.4M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/41.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3200 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [5]:
from transformers import LayoutLMTokenizer

tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")

def encode_example(words, boxes, label, max_length=512):
    encoding = tokenizer(
        words,
        boxes=boxes,
        padding='max_length',
        truncation=True,
        max_length=max_length,
    )
    encoding['labels'] = label
    return encoding

In [6]:
from torch.utils.data import Dataset

class RVLCDIPDataset(Dataset):
    def __init__(self, hf_dataset):
        self.dataset = hf_dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        words = item['ocr_words']
        boxes = item['word_boxes']
        label = item['label']
        encoding = encode_example(words, boxes, label)
        # Remove the squeeze operation
        return encoding

In [8]:
from transformers import LayoutLMForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch

model = LayoutLMForSequenceClassification.from_pretrained(
    "microsoft/layoutlm-base-uncased",
    num_labels=16  # RVL-CDIP has 16 classes
)

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    eval_strategy="steps",
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=5e-5,
    save_total_limit=2,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataset = RVLCDIPDataset(ds['train'])
eval_dataset = RVLCDIPDataset(ds['test'])

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

trainer.train()

Some weights of LayoutLMForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zeusgaming-2006 (zeusgaming-2006-nothing) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Keyword arguments {'boxes': [[522, 876, 530, 928]]} not recognized.
Keyword arguments {'boxes': [[103, 11, 237, 31], [245, 13, 295, 31], [116, 34, 156, 42], [170, 34, 294, 42], [345, 39, 525, 55], [289, 41, 339, 57], [155, 45, 251, 59], [681, 53, 759, 69], [617, 55, 665, 69], [615, 83, 759, 99], [447, 85, 507, 101], [513, 85, 551, 101], [177, 87, 235, 103], [241, 87, 301, 101], [307, 87, 339, 101], [45, 89, 113, 103], [615, 99, 637, 115], [447, 101, 505, 115], [513, 101, 561, 115], [83, 103, 131, 119], [175, 103, 237, 117], [615, 113, 693, 129], [447, 115, 497, 131], [503, 115, 545, 131], [45, 119, 77, 133], [447, 131, 479, 145], [615, 131, 637, 143], [45, 133, 87, 149], [93, 133, 149, 147], [615, 143, 695, 159], [447, 145, 479, 161], [485, 145, 533, 161], [44, 146, 142, 162], [176, 146, 272, 162], [533, 159, 573, 175], [289, 161, 347, 177], [353, 161, 411, 177], [447, 161, 489, 175], [493, 161, 527, 175], [47, 163, 159, 179], [175, 163, 283, 177], [615, 189, 667, 205], [447, 191, 533,

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).